In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import six

import chainer
from chainer import computational_graph
from chainer import cuda
from chainer import optimizers
from chainer import serializers
from chainer import functions as f
from chainer import links as l
from chainer import initializer, cuda
from chainer.functions.loss.vae import gaussian_kl_divergence


In [2]:
np.random.seed(123)

In [88]:
# 遺伝子発現量RNAseqを読み込む
rnaseq_file = os.path.join('data', 'pancan_scaled_zeroone_rnaseq.tsv')
rnaseq_df = pd.read_table(rnaseq_file, index_col=0).astype(np.float32)
print(rnaseq_df.shape)
print(rnaseq_df.values[0].dtype)
rnaseq_df.head(2)

(10459, 5000)
float32


,RPS4Y1,XIST,KRT5,AGR2,CEACAM5,KRT6A,KRT14,CEACAM6,DDX3Y,KDM5D,...,FAM129A,C8orf48,CDK5R1,FAM81A,C13orf18,GDPD3,SMAGP,C2orf85,POU5F1B,CHST2
TCGA-02-0047-01,0.678296,0.289910,0.034230,0.0,0.0,0.084731,0.031863,0.037709,0.746797,0.687832,...,0.440610,0.428782,0.732819,0.634340,0.580662,0.294313,0.458134,0.478219,0.168263,0.638497
TCGA-02-0055-01,0.200633,0.654917,0.181993,0.0,0.0,0.100606,0.050011,0.092586,0.103725,0.140642,...,0.620658,0.363207,0.592269,0.602755,0.610192,0.374569,0.722420,0.271356,0.160465,0.602560


In [89]:
# Split 10% test set randomly
test_set_percent = 0.1
rnaseq_test_df = rnaseq_df.sample(frac=test_set_percent).astype(np.float32)
rnaseq_train_df = rnaseq_df.drop(rnaseq_test_df.index).astype(np.float32)

In [90]:
# Set hyper parameters
original_dim = rnaseq_df.shape[1]
latent_dim = 100

batch_size = 50
epochs = 50
learning_rate = 0.0005

epsilon_std = 1.0
kappa = 1

Variational Auto Encoder

In [148]:
class Xavier(initializer.Initializer):
    """
    Xavier initializaer
    Reference:
    * https://jmetzen.github.io/2015-11-27/vae.html
    * https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    """

    def __init__(self, fan_in, fan_out, constant=1, dtype=None):
        self.fan_in = fan_in
        self.fan_out = fan_out
        self.high = constant*np.sqrt(6.0/(fan_in + fan_out))
        self.low = -self.high
        super(Xavier, self).__init__(dtype)

    def __call__(self, array):
        xp = cuda.get_array_module(array)
        args = {'low': self.low, 'high': self.high, 'size': array.shape}
        if xp is not np:
            # Only CuPy supports dtype option
            if self.dtype == np.float32 or self.dtype == np.float16:
                # float16 is not supported in cuRAND
                args['dtype'] = np.float32
        array[...] = xp.random.uniform(**args)


class VariationalAutoEncoder(chainer.Chain):
    """Variational AutoEncoder"""

    def __init__(self, n_in, n_latent, n_h, act_func=f.tanh):
        super(VariationalAutoEncoder, self).__init__()
        self.act_func = act_func
        with self.init_scope():
            # encoder
            self.le1 = l.Linear(n_in,       n_h,        initialW=Xavier(n_in, n_h))
            self.le2 = l.Linear(n_h,        n_h,        initialW=Xavier(n_h, n_h))
            self.bn1 = l.BatchNormalization(n_h)
            self.le3_mu = l.Linear(n_h,     n_latent,   initialW=Xavier(n_h,  n_latent))
            self.le3_ln_var = l.Linear(n_h, n_latent,   initialW=Xavier(n_h,  n_latent))
            # decoder
            self.ld1 = l.Linear(n_latent,   n_h,        initialW=Xavier(n_latent, n_h))
            self.ld2 = l.Linear(n_h,        n_h,        initialW=Xavier(n_h, n_h))
            self.ld3 = l.Linear(n_h,        n_in,       initialW=Xavier(n_h, n_in))
            self.ld4 = l.Linear(n_latent,   n_in,       initialW=Xavier(n_latent, n_in))

    def __call__(self, x, sigmoid=True):
        """AutoEncoder"""
        return self.decode(self.encode(x)[0], sigmoid)

    def encode(self, x):
        if type(x) != chainer.variable.Variable:
            x = chainer.Variable(x)
        x.name = "x"
        # h1 = self.act_func(l.BatchNormalization(self.le1(x)))
        h1 =self.act_func( self.le1(x))
        h1.name = "enc_h1"
        h1_2 = h1#self.bn1(h1)
        
        h2 = self.act_func(self.le2(h1_2))
        h2.name = "enc_h2"
        mu = self.le3_mu(h2)
        mu.name = "z_mu"
        ln_var = self.le3_ln_var(h2)  # ln_var = log(sigma**2)
        ln_var.name = "z_ln_var"
        return mu, ln_var

    def decode(self, z, sigmoid=True):
        # h1 = self.act_func(self.ld1(z))
        # h1.name = "dec_h1"
        # h2 = self.act_func(self.ld2(h1))
        # h2.name = "dec_h2"
        # h3 = self.ld3(h2)
        # h3.name = "dec_h3"
        h4 = self.ld4(z)
        h4.name = "dec_h4"
        if sigmoid:
            return f.sigmoid(h4)
        else:
            return h4

    def get_loss_func(self, C=1.0, k=1):
        """Get loss function of VAE.
        The loss value is equal to ELBO (Evidence Lower Bound)
        multiplied by -1.
        Args:
            C (int): Usually this is 1.0. Can be changed to control the
                second term of ELBO bound, which works as regularization.
            k (int): Number of Monte Carlo samples used in encoded vector.
        """
        def lf(x):
            mu, ln_var = self.encode(x)
            batch_size = len(mu.data)
            # reconstruction loss
            rec_loss = 0
            for l in range(k):
                z = f.gaussian(mu, ln_var)
                z.name = "z"
                rec_loss += f.bernoulli_nll(x, self.decode(z, sigmoid=True)) \
                    / (k * batch_size)
            self.rec_loss = rec_loss
            self.rec_loss.name = "reconstruction error"
            self.latent_loss = C * gaussian_kl_divergence(mu, ln_var) / batch_size
            self.latent_loss.name = "latent loss"
            self.loss = self.rec_loss + self.latent_loss
            self.loss.name = "loss"
            return self.loss
        return lf

In [149]:
# Prepare VAE model
model = VariationalAutoEncoder(original_dim, latent_dim, latent_dim, act_func=f.relu)

In [150]:
xp = np
# Setup optimizer
optimizer = optimizers.Adam(alpha=learning_rate)
optimizer.setup(model)

In [151]:
# N = rnaseq_train_df.shape[0]
# N_test = rnaseq_test_df.shape[0]
N = 9000
x_train, x_test = np.split(rnaseq_df,   [N])
print(x_train.values[0].dtype)

float32


In [152]:
N_test = x_test.size
print(N_test)

7295000


In [154]:
# import time
# total_losses = np.zeros(epochs, dtype=np.float32)

for epoch in six.moves.range(1, epochs + 1):
    print('epoch', epoch)
    # epoch_time = time.time()
    
    # training
    perm = np.random.permutation(N)
    sum_loss = 0       # total loss
    sum_rec_loss = 0   # reconstruction loss
    
    for i in six.moves.range(0, N, batch_size):
        x = chainer.Variable(xp.asarray(x_train.values[perm[i:i + batch_size]]))
        optimizer.update(model.get_loss_func(), x)
        sum_loss += float(model.loss.data) * len(x.data)
        sum_rec_loss += float(model.rec_loss.data) * len(x.data)
    
    print('train mean loss={}, mean reconstruction loss={}'
          .format(sum_loss / N, sum_rec_loss / N))
    
#     # evaluation
#     sum_loss = 0
#     sum_rec_loss = 0
#     with chainer.no_backprop_mode():
#         for i in six.moves.range(0, N_test, batch_size):
#             x = chainer.Variable(xp.asarray(x_test.values[i:i + batch_size]))
#             loss_func = model.get_loss_func(k=10)
#             loss_func(x)
#             sum_loss += float(model.loss.data) * len(x.data)
#             sum_rec_loss += float(model.rec_loss.data) * len(x.data)
#             del model.loss
#     print('test  mean loss={}, mean reconstruction loss={}'
#           .format(sum_loss / N_test, sum_rec_loss / N_test))


epoch 1
train mean loss=3496.5251559787325, mean reconstruction loss=3465.932923719618
epoch 2
train mean loss=3481.5491644965277, mean reconstruction loss=3457.869504123264
epoch 3
train mean loss=3471.83759765625, mean reconstruction loss=3450.4716851128474
epoch 4
train mean loss=3465.675351291233, mean reconstruction loss=3446.482469346788
epoch 5
train mean loss=3461.988062879774, mean reconstruction loss=3444.2850992838544
epoch 6
train mean loss=3458.95608859592, mean reconstruction loss=3442.571861436632
epoch 7
train mean loss=3456.8594767252603, mean reconstruction loss=3440.8762953016494
epoch 8
train mean loss=3454.1037651909724, mean reconstruction loss=3438.6315199110245
epoch 9
train mean loss=3452.1212239583333, mean reconstruction loss=3436.969173177083
epoch 10
train mean loss=3450.7045925564234, mean reconstruction loss=3435.912411838108
epoch 11
train mean loss=3449.8045328776043, mean reconstruction loss=3435.1784152560763
epoch 12
train mean loss=3448.364910210503

In [10]:
# Learning loop
loss_list_train = []
loss_list_test = []
N=600000

for epoch in six.moves.range(1, epochs + 1):
    print('epoch', epoch)

    # training
    perm = np.random.permutation(N)
    sum_loss = 0       # total loss
    sum_rec_loss = 0   # reconstruction loss
    
    for t in xrange(num_trains_per_epoch):
    
    
    for i in six.moves.range(0, N, batch_size):
        x = chainer.Variable(xp.asarray(rnaseq_train_df[perm[i:i + batchsize]]))
        optimizer.update(model.get_loss_func(), x)
        if epoch == 1 and i == 0 and graph_gen:
            graph_export(model)

        sum_loss += float(model.loss.data) * len(x.data)
        sum_rec_loss += float(model.rec_loss.data) * len(x.data)
    loss_train = sum_loss / N
    print('train mean loss={}, mean reconstruction loss={}'.format(sum_loss / N, sum_rec_loss / N))

    # evaluation
    sum_loss = 0
    sum_rec_loss = 0
    with chainer.no_backprop_mode():
        for i in six.moves.range(0, N_test, batchsize):
            x = chainer.Variable(xp.asarray(x_test[i:i + batchsize]))
            loss_func = model.get_loss_func(k=10)
            loss_func(x)
            sum_loss += float(model.loss.data) * len(x.data)
            sum_rec_loss += float(model.rec_loss.data) * len(x.data)
            del model.loss
    print('test  mean loss={}, mean reconstruction loss={}'
          .format(sum_loss / N_test, sum_rec_loss / N_test))

epoch 1


NameError: name 'x_train' is not defined